In [ ]:
import json
from IPython.display import display, Javascript
from luxai_s3.wrappers import LuxAIS3GymEnv, RecordEpisode
import time
from proxy_agent import ProxyAgent

def render_episode(episode: RecordEpisode) -> None:
    data = json.dumps(episode.serialize_episode_data(), separators=(",", ":"))
    display(Javascript(f"""
var iframe = document.createElement('iframe');
iframe.src = 'https://s3vis.lux-ai.org/#/kaggle';
iframe.width = '100%';
iframe.scrolling = 'no';

iframe.addEventListener('load', event => {{
    event.target.contentWindow.postMessage({data}, 'https://s3vis.lux-ai.org');
}});

new ResizeObserver(entries => {{
    for (const entry of entries) {{
        entry.target.height = `${{Math.round(320 + 0.3 * entry.contentRect.width)}}px`;
    }}
}}).observe(iframe);

element.append(iframe);
    """))

def evaluate_agents(agent_1_cls, agent_2_cls, seed=42, games_to_play=3, replay_save_dir="replays", render=True, model_name=None):
    env = RecordEpisode(
        LuxAIS3GymEnv(numpy_output=True), save_on_close=True, save_on_reset=True, save_dir=replay_save_dir
    )
    obs, info = env.reset(seed=seed)
    total_wins = np.zeros((2))
    for i in range(games_to_play):
        start = time.time()
        obs, info = env.reset(seed=seed+i)
        env_cfg = info["params"] # only contains observable game parameters
        player_0 = agent_1_cls("player_0", env_cfg, model_name=model_name)
        player_1 = agent_2_cls("player_1", env_cfg)
    
        # main game loop
        game_done = False
        step = 0
        print(f"Running game {i}")
        while not game_done:
            actions = dict()
            
            for agent in [player_0, player_1]:
                actions[agent.player] = agent.act(step=step, obs=obs[agent.player])
            obs, reward, terminated, truncated, info = env.step(actions)
            # info["state"] is the environment state object, you can inspect/play around with it to e.g. print
            # unobservable game data that agents can't see
            dones = {k: terminated[k] | truncated[k] for k in terminated}
            if dones["player_0"] or dones["player_1"]:
                game_done = True
            step += 1
        total_wins += np.array([reward["player_0"],reward["player_1"]])
        if render:
            render_episode(env)
        print("Runtime: ", time.time()-start)
    print(total_wins/games_to_play)
    env.close() # free up resources and save final replay

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
np.set_printoptions(linewidth=2000)


In [ ]:
def reconstruct_path(came_from, came_from_flag, current):
    total_path = [current]
    while came_from_flag[current[0], current[1]]==1:
        current = [int(came_from[current[0],current[1],0]),int(came_from[current[0],current[1],1])]
        total_path.insert(0,current)
    return total_path

def h(start, goal, move_cost):
    steps = abs(goal[0]-start[0]) + abs(goal[1]-start[0])
    return steps*move_cost

def get_neighbors(node):
    neighbors = [[node[0],node[1]-1],
                [node[0]+1,node[1]],
                [node[0],node[1]+1],
                [node[0]-1,node[1]],]
    neighbors = np.unique(np.clip(np.array(neighbors),0,23),axis=0)
    neighbors = neighbors[(neighbors[:,:]!=np.array(node)).any(1)]
    return neighbors.tolist()
    
def find_lowest(f_score, open_set):
    index = None
    lowest = 1e20
    for ii, node in enumerate(open_set):
        if f_score[node[0],node[1]]<lowest:
            index = ii
            lowest = f_score[node[0],node[1]]
    return index
    
def a_star(start, goal, tile_map, energy_map, frag_map, move_cost, nebula_drain, use_energy=True, budget=100):
    asteroids = np.zeros((24,24))
    asteroids[tile_map==2] = asteroids[tile_map==2] + 1e6
    nebulas = np.zeros((24,24))
    nebulas[tile_map==1] = nebulas[tile_map==1] + nebula_drain
    #energy = -energy_map + np.full((24,24),np.max(energy_map))
    energy_max = np.max(energy_map)
    moves = np.full((24,24), move_cost)
    moves[frag_map==1] = 0
    budget = budget
    def d_move(pos):
        return asteroids[pos[0],pos[1]] + moves[pos[0],pos[1]]
    def d_energy(pos, adjust):
        if adjust:
            return nebulas[pos[0],pos[1]] - energy_map[pos[0],pos[1]] + energy_max + nebulas[pos[0],pos[1]]
        else:
            return nebulas[pos[0],pos[1]] - energy_map[pos[0],pos[1]] + nebulas[pos[0],pos[1]]

    def d(pos, use_energy):
        if use_energy:
            return d_move(pos) + d_energy(pos, use_energy)
        else:
            return d_move(pos)
            
    open_set = [start]
    came_from = -np.ones((24,24,2))
    came_from_flag = np.zeros((24,24))
    #came_from_flag[start[0], start[1]] = 1
    
    g_score = np.full((24,24),np.inf)
    g_score[start[0],start[1]] = 0    
    g_score_budget = np.full((24,24),np.inf)
    g_score_budget[start[0],start[1]] = 0
    
    f_score = np.full((24,24),np.inf)
    f_score[start[0],start[1]] = h(start, goal, move_cost)
    #if not use_energy:
    #    print("start")
    while open_set:
        lowest = find_lowest(f_score, open_set)
        current = open_set.pop(lowest)
        #print(current)
        if current[0]==goal[0] and current[1]==goal[1]:
            #if not use_energy:
            #    print("end succ", g_score_budget[current[0],current[1]], g_score[current[0],current[1]])
            return reconstruct_path(came_from, came_from_flag, current), g_score[current[0], current[1]]
        for neighbor in get_neighbors(current):
            temp_g_score = g_score[current[0],current[1]] + d([neighbor[0],neighbor[1]], use_energy=use_energy)
            if temp_g_score<g_score[neighbor[0],neighbor[1]]:
                came_from[neighbor[0],neighbor[1],0] = current[0]
                came_from[neighbor[0],neighbor[1],1] = current[1]
                came_from_flag[neighbor[0],neighbor[1]] = 1
                g_score[neighbor[0],neighbor[1]] = temp_g_score
                if not use_energy:
                    g_score_budget[neighbor[0],neighbor[1]] = g_score_budget[current[0],current[1]] + d_move([neighbor[0],neighbor[1]]) + d_energy([neighbor[0],neighbor[1]], False)
                    #print(g_score_budget[neighbor[0],neighbor[1]], g_score[neighbor[0],neighbor[1]])
                f_score[neighbor[0],neighbor[1]] = temp_g_score + h(neighbor,goal,move_cost)
                if not use_energy and g_score_budget[neighbor[0],neighbor[1]]>budget:
                    #print("out of energy")
                    pass
                else:
                    if neighbor not in open_set:
                        open_set.append(neighbor)
    return [[0,0]], 1e10 

In [ ]:
#%%writefile my_agent_2.0/agent.py
from my_agent.lux.utils import direction_to, direction_to_change
import matplotlib.pyplot as plt
#from agent.maps import RelicMap
import numpy as np
import random
from my_agent_v2.maps import EnergyMap, RelicMap, TileMap
from old_agents import Agent, Agent2, Agent3
#np.set_printoptions(linewidth=200)

class Agent4():
    def __init__(self, player: str, env_cfg) -> None:
        self.player = player
        self.opp_player = "player_1" if self.player == "player_0" else "player_0"
        self.team_id = 0 if self.player == "player_0" else 1
        self.opp_team_id = 1 if self.team_id == 0 else 0
        np.random.seed(0)
        self.env_cfg = env_cfg
        if self.player=="player_0":
            self.start_pos = [0,0]
            self.pnum = 1
        else:
            self.start_pos = [23,23]
            self.pnum = 0
        self.unit_explore_locations = dict()
        self.relic_node_positions = []
        self.discovered_relic_nodes_ids = set()
        self.n_units = self.env_cfg["max_units"]
        self.match_num = 1
        self.relic_map = RelicMap(self.n_units)
        self.tile_map = TileMap()
        self.energy_map = EnergyMap()
        self.move_cost = 3.0
        self.nebula_drain = 5.0
        self.move_check = 0
        self.nebula_check = 0
        
        self.range = self.env_cfg["unit_sensor_range"]
        self.sap_range = self.env_cfg["unit_sap_range"]
        self.sap_cost = self.env_cfg["unit_sap_cost"]
        self.width = self.env_cfg["map_width"]
        self.height = self.env_cfg["map_height"]
        a = np.stack((np.repeat(np.arange(24),24,axis=0).reshape((24,24)), np.repeat(np.arange(24),24,axis=0).reshape((24,24)).T),axis=2)
        self.explore_choices = a[np.sum(np.abs(a-np.array(self.start_pos)),axis=-1)<24-self.range]
        self.explore_targets = []
        a = [[i,abs(i-23+self.range)] for i in range(23-self.range+1)]
        if self.team_id==1:
            a = [[abs(t[1]-23),abs(t[0]-23)] for t in a]
        for ii, t in enumerate(a):
            if ii%(self.range+2)==0:
                self.explore_targets.append(t)
        self.relic_targets = []
        self.fragment_targets = []
        self.locked_relic_targets = []
        self.locked_fragment_targets = []
        self.fragment_locations = []
        self.occupied_fragments = []

        self.n_explore_units = 0
        self.unit_has_target = -np.ones((self.n_units)) # -1=no target; 0=explore target; 1=relic target; 2=on relic
        self.unit_targets = dict(zip(range(0,self.n_units), np.zeros((self.n_units,2))))
        self.unit_path = dict(zip(range(0,self.n_units), [[] for i in range(0,self.n_units)]))
        self.unit_moved = np.zeros((self.n_units))
        self.prev_points = 0
        self.prev_points_increase = 0
        self.prev_actions = None
        self.previous_energys = 100*np.zeros((self.n_units))
        self.previous_positions = -np.ones((self.n_units,2))


    def get_explore(self, current):
        a = np.stack((np.repeat(np.arange(24),24,axis=0).reshape((24,24)), np.repeat(np.arange(24),24,axis=0).reshape((24,24)).T),axis=2)
        a[current!=-1] = [100,100]
        self.explore_choices = a[np.sum(np.abs(a-np.array(self.start_pos)),axis=-1)<24-self.range].tolist()
        if self.explore_choices:
            return random.choice(self.explore_choices)
        else:
            x = np.random.randint(0,24)
            y = np.random.randint(0,24-x)
            return [abs(x-self.start_pos[0]), abs(y-self.start_pos[1])]

    def get_moves(self, obs, unit_id, unit_pos):
        prev_pos = [unit_pos[0] - direction_to_change(self.prev_actions[unit_id][0])[0], unit_pos[1] - direction_to_change(self.prev_actions[unit_id][0])[1]]
        new_pos = [[unit_pos[0], unit_pos[1]-1],
                  [unit_pos[0]+1, unit_pos[1]],
                  [unit_pos[0], unit_pos[1]+1],
                  [unit_pos[0]-1, unit_pos[1]]]
        moves = [0]
        for ii, pos in enumerate(new_pos):
            if pos[0]<0 or pos[1]<0 or pos[0]>=self.width or pos[1]>=self.height or (pos[0]==prev_pos[0] and pos[1]==prev_pos[1]) or obs["map_features"]["tile_type"][pos[0], pos[1]]==2 :
            #if pos[0]<0 or pos[1]<0 or pos[0]>23 or pos[1]>23 or obs["map_features"]["tile_type"][pos[0], pos[1]]==2:
                pass
            else:
                moves.append(direction_to(unit_pos, pos))
        #print(moves)
        return moves
        
    # moves around asteroids
    def move_obstacle_avoid(self, obs, unit_id, unit_pos, direction):
        moves = self.get_moves(obs, unit_id, unit_pos)
        if direction in moves:
            return direction
        elif moves:
            return random.choice(moves)
        else:
            return 0
            
    def relic_to_targets(self, pos):
        targets = []
        for i in range(-2,3,1):
            for j in range(-2,3,1):
                if pos[0]+i>=0 and pos[0]+i<=23 and pos[1]+j>=0 and pos[1]+j<=23:
                    new_target = np.array([pos[0]+i, pos[1]+j])
                    mirrored_target = np.abs(new_target-np.array([self.width, self.height]))
                    targets.append(new_target)
                    #targets.append(mirrored_target)
        return targets
        
    def reset(self):
        self.match_num += 1
        self.relic_map.reset()
        self.explore_targets = []
        self.unit_has_target = -np.ones((self.n_units)) # -1=no target; 0=explore target; 1=relic target; 2=on relic, 3=known fragment
        self.unit_targets = dict(zip(range(0,self.n_units), np.zeros((self.n_units,2))))
        self.unit_path = dict(zip(range(0,self.n_units), [[] for i in range(0,self.n_units)]))
        self.unit_moved = np.zeros((self.n_units))
        self.prev_points = 0
        self.prev_points_increase = 0
        self.prev_actions = np.zeros((self.env_cfg["max_units"], 3), dtype=int)
        self.prev_energys = 100*np.ones((self.n_units))
        self.previous_positions = -np.ones((self.n_units,2))
        self.occupied_fragments = []
        self.n_explore_units = 0
        if self.match_num==2:
            self.n_explore_units = 3
        elif self.match_num==3:
            self.n_explore_units = 2

    def find_best_unit(self, goal, available_unit_ids, unit_positions, use_energy=True):
        best_unit = 0
        best_pos = [-1,-1]
        best_dist = np.inf
        best_path = [goal]
        if len(available_unit_ids)>0:
            for ii, unit_id in enumerate(available_unit_ids):
                unit_pos = unit_positions[unit_id]
                path, dist = a_star(unit_pos, goal, self.tile_map.map, self.energy_map.map, self.relic_map.map_knowns, self.move_cost, self.nebula_drain, use_energy=use_energy)
                if dist<best_dist:
                    best_dist = dist
                    best_path = path
                    best_unit = unit_id
                    best_pos = unit_pos
        return best_unit, best_path

    def compare_positions(self, pos1, pos2):
        return pos1[0]==pos2[0] and pos1[1]==pos2[1]

    def get_attack_targets(self):
        fragments = self.relic_map.get_fragments(self.start_pos)
        targets = []
        for frag in fragments:
            if self.tile_map.map[frag[0],frag[1]]!=2:
                if self.start_pos[0]==0:
                    if frag[0]+frag[1]>23:
                        if frag in self.occupied_fragments: 
                            targets.append(frag)
                        else:
                            targets.insert(0,frag)
                else:
                    if frag[0]+frag[1]<23:
                        if frag in self.occupied_fragments: 
                            targets.append(frag)
                        else:
                            targets.insert(0,frag)
        return targets

    def get_defend_targets(self, pos):
        #pos = self.occupied_fragments[unit%len(self.occupied_fragments)]
        surrounding = []
        distances = []
        for x in range(-2,3,1):
            for y in range(-2,3,1):
                if (x!=0 or y!=0) and pos[0]+x>=0 and pos[1]+y>=0 and pos[0]+x<=23 and pos[1]+y<=23:
                    new_pos = [pos[0]+x,pos[1]+y]
                    surrounding.append(new_pos)
                    distances.append(abs(new_pos[0]-abs(self.start_pos[0]-23))+abs(new_pos[1]-abs(self.start_pos[1]-23)))
        surrounding = np.array(surrounding)[np.argsort(np.array(distances))]
        surrounding = surrounding[:8].tolist()
        best_e = -np.inf
        best_pos = [-1,-1]
        for s in surrounding:
            if self.energy_map.map[s[0],s[1]]>best_e:
                best_e = self.energy_map.map[s[0],s[1]]
                best_pos = s
        return best_pos
        

    def get_enemy_targets(self, pos, enemy_positions, relative=True):
        targets = []
        for dx in range(-self.sap_range,self.sap_range):
            for dy in range(-self.sap_range,self.sap_range):
                if [pos[0]+dx,pos[1]+dy] in enemy_positions:
                    if relative:
                        targets.append([dx,dy])
                    else:
                        targets.append([pos[0]+dx,pos[1]+dy])
        return targets
        
    def repath(self, unit_positions):
        for unit in range(self.n_units):
            pos = unit_positions[unit]
            if self.compare_positions(pos, [-1,-1]):
                pos = self.start_pos
            if self.unit_path[unit]:
                self.unit_path[unit],_ = a_star(pos, self.unit_targets[unit], self.tile_map.map, self.energy_map.map, self.move_cost, self.nebula_drain)
                self.unit_path[unit].pop(0)

    def free_target(self, unit, pos):
        if self.unit_has_target[unit]==1:
            self.relic_targets.append([int(self.unit_targets[unit][0]),int(self.unit_targets[unit][1])])
        if self.unit_has_target[unit]==2:
            self.fragment_targets.append([int(self.unit_targets[unit][0]),int(self.unit_targets[unit][1])])
            if [int(self.unit_targets[unit][0]),int(self.unit_targets[unit][1])] in self.occupied_fragments:
                self.occupied_fragments.remove([int(self.unit_targets[unit][0]),int(self.unit_targets[unit][1])])

    def sort_targets(self, targets, positions):
        
        if targets and positions:
                T = np.repeat(np.expand_dims(np.array(targets),axis=0),len(positions),axis=0)
                P = np.tile(np.array(positions), len(targets)).reshape(len(positions),len(targets),2)
                dists = np.sum(np.abs(T-P),axis=2)
                best = np.min(dists,axis=0)
                return (np.array(targets)[np.argsort(best)]).tolist()
        else:
            return []

    def preaim(self, pos):
        frags = self.relic_map.get_fragments(self.start_pos)
        if pos in frags:
            return pos
        frags = self.relic_map.get_fragments(self.start_pos, own=True)
        dist = np.sum(np.abs(np.array(frags)-np.array(pos)),axis=-1)
        target = frags[np.argsort(dist)[0]]
        path, _ = a_star(pos, target, self.tile_map.map, self.energy_map.map, self.relic_map.map_knowns, self.move_cost, self.nebula_drain, use_energy=True)
        return path[1]

    # bunnyhop mechanic (maximize points by avoiding doubling on fragment)
    def bunnyhop(self, unit, unit_positions):
        counter = 0
        unit_pos = unit_positions[unit]
        for unit2 in range(self.n_units):            
            if self.unit_has_target[unit2]==2 and self.tile_map.map[unit_positions[unit2][0],unit_positions[unit2][1]]!=2 and len(self.unit_path[unit])>1 and self.compare_positions(self.unit_path[unit][0],unit_positions[unit2]):
                self.unit_path[unit2] = self.unit_path[unit][1:]
                self.unit_targets[unit2] = self.unit_targets[unit]
                self.unit_has_target[unit2] = 1#self.unit_has_target[unit]
                self.unit_path[unit] = [unit_positions[unit2]]
                self.unit_targets[unit] = unit_positions[unit2]
                self.unit_has_target[unit] = 1
                counter +=1
                if counter<10:
                    self.bunnyhop(unit2, unit_positions)
    
    def act(self, step: int, obs, remainingOverageTime: int = 60):
        """implement this function to decide what actions to send to each available unit. 
        
        step is the current timestep number of the game starting from 0 going up to max_steps_in_match * match_count_per_episode - 1.
        """
        #print("Step: ", step)
        unit_mask = np.array(obs["units_mask"][self.team_id]) # shape (max_units, )
        unit_positions = np.array(obs["units"]["position"][self.team_id]) # shape (max_units, 2)
        enemy_positions = []
        for pos in np.array(obs["units"]["position"][abs(self.team_id-1)]).tolist():
            if pos[0]!=-1 and pos[1]!=-1:
                enemy_positions.append(pos)
        unit_energys = np.array(obs["units"]["energy"][self.team_id]) # shape (max_units, 1)
        observed_relic_node_positions = np.array(obs["relic_nodes"]) # shape (max_relic_nodes, 2)
        observed_relic_nodes_mask = np.array(obs["relic_nodes_mask"]) # shape (max_relic_nodes, )
        team_points = np.array(obs["team_points"]) # points of each team, team_points[self.team_id] is the points of the your team
        increase = team_points[self.team_id]-self.prev_points
        # ids of units you can control at this timestep
        actions = np.zeros((self.env_cfg["max_units"], 3), dtype=int)
        current_tile_map = obs["map_features"]["tile_type"]
        current_energy_map = obs["map_features"]["energy"]
        
        
        if step in [102,203,304,405]:
            self.reset()
            
        # visible relic nodes
        visible_relic_node_ids = set(np.where(observed_relic_nodes_mask)[0])
        # save any new relic nodes that we discover for the rest of the game.
        for ii in visible_relic_node_ids:
            if ii not in self.discovered_relic_nodes_ids:
                # explore units switch to relic collection
                self.relic_map.new_relic(observed_relic_node_positions[ii])
                self.unit_has_target[self.unit_has_target==0]=-1
                self.unit_has_target[self.unit_has_target==3]=-1
                self.discovered_relic_nodes_ids.add(ii)
                self.discovered_relic_nodes_ids.add((ii+3)%6)
                self.relic_node_positions.append(observed_relic_node_positions[ii])
                #self.relic_targets.extend(self.relic_to_targets(observed_relic_node_positions[ii]))
                self.relic_targets.extend(self.relic_map.get_possibles(self.start_pos, own=True))
                # remove duplicates from relic targets
                self.relic_targets = np.array(list({array.tobytes(): array for array in np.array(self.relic_targets)}.values())).tolist()
                #print(step, self.discovered_relic_nodes_ids, "\n", self.relic_targets, "\n",  self.unit_has_target, "\n",  self.unit_targets)
        # update maps
        available_unit_ids = np.where(unit_mask)[0].tolist()
        self.relic_map.step(unit_positions, increase)
        tile_shift = self.tile_map.update(current_tile_map)
        energy_shift = self.energy_map.update(current_energy_map)        

        # find out move cost
        if step>2 and not self.move_check and self.tile_map.map[unit_positions[0][0],unit_positions[0][1]]!=1 and self.unit_moved[0]:
            self.move_cost=self.previous_energys[0]-unit_energys[0]+self.energy_map.map[unit_positions[0][0],unit_positions[0][1]]
            self.move_check=1
        # find out nebula drain
        if not self.nebula_check and self.move_check:
            for unit in available_unit_ids:
                if self.unit_moved[unit] and  self.tile_map.map[unit_positions[unit][0],unit_positions[unit][1]]==1:
                    self.nebula_check=1
                    self.nebula_drain = -(unit_energys[unit]-self.previous_energys[unit]-self.energy_map.map[unit_positions[unit][0],unit_positions[unit][1]]+self.move_cost)
                    break
            
        if tile_shift or energy_shift:
            self.fragment_targets.extend(self.locked_fragment_targets)
            self.relic_targets.extend(self.locked_relic_targets)
            self.locked_relic_targets = []
            self.locked_fragment_targets = []

        # collision detection ->repath
        for unit in available_unit_ids:
            if self.prev_actions[unit][0] in [1,2,3,4] and self.compare_positions(self.previous_positions[unit], unit_positions[unit]) and self.previous_energys[unit]>self.move_cost:
                    self.unit_path[unit],_ = a_star(unit_positions[unit], self.unit_targets[unit], self.tile_map.map, self.energy_map.map, self.relic_map.map_knowns, self.move_cost, self.nebula_drain, use_energy=True)
                    self.unit_path[unit].pop(0)
        
        self.occupied_fragments = []
        for unit in available_unit_ids.copy():
            pos = [int(unit_positions[unit][0]), int(unit_positions[unit][1])]
            # set moved flag necessary for kill detection
            if not self.compare_positions(pos, self.start_pos):
                self.unit_moved[unit] = 1
            # if unit is on unoccupied fragment, stay and remove this tile as target from other units, but append to possible/fragment targets if necessary
            if self.relic_map.map_knowns[pos[0], pos[1]]==1 and [pos[0],pos[1]] not in self.occupied_fragments and self.compare_positions(self.unit_targets[unit],pos):
                self.unit_has_target[unit] = 2
                self.unit_targets[unit] = pos
                self.unit_path[unit] = []
                self.occupied_fragments.append([int(pos[0]), int(pos[1])])
                remain_units = available_unit_ids.copy()
                remain_units.remove(unit)
                for unit2 in remain_units:
                    if self.unit_targets[unit2][0]==pos[0] and self.unit_targets[unit2][1]==pos[1]:# and self.unit_has_target[unit]!=3:
                        # decide who goes and who stays based on energy
                        if unit_energys[unit]>unit_energys[unit2]:
                            self.unit_has_target[unit2] = -1
                        else:
                            self.unit_has_target[unit2] = 2
                            self.unit_has_target[unit] = -1
                            self.unit_targets[unit2] = pos
                            self.unit_path[unit2] = []
            # remove target if possible fragment has been cleared (by other unit)
            if self.unit_has_target[unit]==1 and self.relic_map.map_possibles[self.unit_targets[unit][0], self.unit_targets[unit][1]]==0 and self.relic_map.map_knowns[self.unit_targets[unit][0], self.unit_targets[unit][1]]!=1:
                self.unit_has_target[unit] = -1
            # retarget def units
            if self.unit_has_target[unit]==3 and self.compare_positions(self.unit_targets[unit],pos):
                if self.fragment_locations:
                    target = self.get_defend_targets(self.fragment_locations[unit%len(self.fragment_locations)])
                    self.unit_targets[unit] = target
                    self.unit_path[unit],_ = a_star(pos, target, self.tile_map.map, self.energy_map.map, self.relic_map.map_knowns, self.move_cost, self.nebula_drain, use_energy=True)
                    self.unit_path[unit].pop(0)
                else:
                    self.unit_has_target[unit] = -1
            # free unit that has been killed and reissue targets if necessary
            if self.unit_moved[unit] and self.compare_positions(pos, self.start_pos):
                self.free_target(unit, pos)
                self.unit_has_target[unit] = -1
                self.unit_moved[unit] = 0
            # untarget if target is asteroid, keep track of blocked targets to reuse when shift happens
            if self.tile_map.map[int(self.unit_targets[unit][0]),int(self.unit_targets[unit][1])]==2 and not self.compare_positions(pos,self.unit_targets[unit]):
                if self.unit_has_target[unit]==1:
                    self.locked_relic_targets.append([int(self.unit_targets[unit][0]),int(self.unit_targets[unit][1])])
                if self.unit_has_target[unit]==2:
                    self.locked_fragment_targets.append([int(self.unit_targets[unit][0]),int(self.unit_targets[unit][1])])
                self.unit_has_target[unit]=-1
            # remove if path empty (can happen if blocked by asteroid or pathing mistakes happens)
            if not self.unit_path[unit] and self.unit_has_target[unit]<2:
                if self.unit_has_target[unit]==0:
                    self.n_explore_units += 1
                self.unit_has_target[unit]=-1
                if not self.compare_positions(pos, self.unit_targets[unit]):
                    self.free_target(unit, pos)
            # if unit has possible or known as target or no energy remove from available
            if self.unit_has_target[unit]==2 or unit_energys[unit]<self.move_cost:
                available_unit_ids.remove(unit)
            if unit_energys[unit]<self.move_cost and not self.compare_positions(pos, self.unit_targets[unit]):
                self.free_target(unit, pos)
            
        self.fragment_targets = self.relic_map.get_fragments(self.start_pos, own=True)
        self.relic_targets = self.relic_map.get_possibles(self.start_pos, own=True)
        for f in self.fragment_targets.copy():
            if self.tile_map.map[f[0],f[1]]==2 or f in self.occupied_fragments:
                self.fragment_targets.remove(f)
        for r in self.relic_targets.copy():
            if self.tile_map.map[r[0],r[1]]==2:
                self.relic_targets.remove(r)
         
        #print(step, "0")
        positions = []
        for u in available_unit_ids:
            positions.append(unit_positions[u])
        all_targets = self.sort_targets(self.fragment_targets, positions) + self.sort_targets(self.relic_targets, positions) 

        for ii, goal in enumerate(all_targets):
            if available_unit_ids:
                unit, path = self.find_best_unit(goal, available_unit_ids, unit_positions, use_energy=False)
                available_unit_ids.remove(unit)
                self.unit_path[unit] = path[1:]
                self.unit_has_target[unit] = 1
                self.unit_targets[unit] = goal
        for ii, goal in enumerate(self.explore_targets):
            if available_unit_ids:
                unit, path = self.find_best_unit(goal, available_unit_ids, unit_positions, use_energy=True)
                available_unit_ids.remove(unit)
                self.unit_path[unit] = path[1:]
                self.unit_has_target[unit] = 0
                self.unit_targets[unit] = goal
                self.explore_targets.remove(goal)
        #if step>300:
        #    print("Step: ", step, "\n", self.unit_has_target, "\n", self.unit_targets, "\n", self.unit_path, "\n", actions, "\n", self.fragment_targets, "\n", all_targets)
        # only keep targets that aren't exploring
        for unit in available_unit_ids.copy():
            if self.unit_has_target[unit]==0 or self.unit_has_target[unit]==3:
                if not self.compare_positions(unit_positions[unit], self.unit_targets[unit]):
                    available_unit_ids.remove(unit)

        #print(step, "1")
        if available_unit_ids:
            attack_targets = self.get_attack_targets()
        # send remaining units to explore (first match) or target 
        for unit in available_unit_ids.copy():
            if not self.n_explore_units>0 and attack_targets:
                #target_id, path = self.find_best_unit(unit_positions[unit], np.arange(len(attack_targets)), attack_targets)
                #if path:
                #    self.unit_path[unit] = path[::-1]
                #self.unit_path[unit].append(attack_targets[0])
                path, _ = a_star(unit_positions[unit], attack_targets[0], self.tile_map.map, self.energy_map.map, self.relic_map.map_knowns, self.move_cost, self.nebula_drain, use_energy=True)
                self.unit_path[unit] = path[1:]
                self.unit_has_target[unit] = 3
                self.unit_targets[unit] = attack_targets[0]
            else:
                target = self.get_explore(current_tile_map)
                path, _ = a_star(unit_positions[unit], target, self.tile_map.map, self.energy_map.map, self.relic_map.map_knowns, self.move_cost, self.nebula_drain, use_energy=True)
                available_unit_ids.remove(unit)
                self.unit_path[unit] = path[1:]
                self.unit_has_target[unit] = 0
                self.unit_targets[unit] = target
                self.n_explore_units -= 1
            
            
            '''for unit in available_unit_ids:
                if self.fragment_locations:
                    target = self.get_defend_targets(self.fragment_locations[unit%len(self.fragment_locations)])
                    self.unit_has_target[unit] = 3
                    self.unit_targets[unit] = target
                    self.unit_path[unit],_ = a_star(pos, target, self.tile_map.map, self.energy_map.map, self.move_cost, self.nebula_drain)
                    self.unit_path[unit].pop(0)'''
        #if step>101 and step<120:
        #    print("Step: ", step, "\n", self.unit_has_target, "\n", self.unit_targets, "\n", self.unit_path, "\n", actions)
        #print(step, "2")
        discover_flag = 0
        # Decide on action. Follow path, if multiple units want to move to possible fragment only let one through, if attacking fire on enemy instead of moving
        for unit in range(self.n_units):
            unit_pos = unit_positions[unit]
            self.bunnyhop(unit, unit_positions)
            # bunnyhop mechanic (maximize points by avoiding doubling on fragment)
            '''for unit2 in range(self.n_units):            
                if self.unit_has_target[unit2]==2 and self.tile_map.map[unit_positions[unit2][0],unit_positions[unit2][1]]!=2 and len(self.unit_path[unit])>1 and self.compare_positions(self.unit_path[unit][0],unit_positions[unit2]):
                    self.unit_path[unit2] = self.unit_path[unit][1:]
                    self.unit_targets[unit2] = self.unit_targets[unit]
                    self.unit_has_target[unit2] = 1#self.unit_has_target[unit]
                    self.unit_path[unit] = [unit_positions[unit2]]
                    self.unit_targets[unit] = unit_positions[unit2]
                    self.unit_has_target[unit] = 1
                    #print("Step: ", step, "swap unit ", unit, " and unit ", unit2, unit_positions[unit2], self.unit_path[unit2])
                    #print("k")
                    for unit3 in range(self.n_units):
                        if self.unit_has_target[unit3]==2 and len(self.unit_path[unit2])>1 and self.compare_positions(self.unit_path[unit2][0],unit_positions[unit3]):
                            self.unit_path[unit3] = self.unit_path[unit2][1:]
                            self.unit_targets[unit3] = self.unit_targets[unit2]
                            self.unit_has_target[unit3] = 1#self.unit_has_target[unit2]
                            self.unit_path[unit2] = [unit_positions[unit3]]
                            self.unit_targets[unit2] = unit_positions[unit3]
                            self.unit_has_target[unit2] = 1'''
        for unit in range(self.n_units):
            unit_pos = unit_positions[unit]
            if unit_mask[unit]:
                for node in self.unit_path[unit]:
                    if self.tile_map.map[node[0],node[1]]==2 or (self.nebula_drain==25 and self.tile_map.map[node[0],node[1]]==1):
                        path, _ = a_star(unit_positions[unit], self.unit_targets[unit], self.tile_map.map, self.energy_map.map, self.relic_map.map_knowns, self.move_cost, self.nebula_drain, use_energy=True)
                        self.unit_path[unit] = path[1:]
                        break
                #for node in self.unit_path[unit]:
                #    if self.tile_map.map[node[0],node[1]]==2 or (self.nebula_drain==25 and self.tile_map.map[node[0],node[1]]==1):
                #        path, _ = a_star(unit_pos, self.unit_targets[unit], self.tile_map.map, self.energy_map.map, self.relic_map.map_knowns, self.move_cost, self.nebula_drain, use_energy=True)
                #        self.unit_path[unit] = path[1:]
                #        break
                #if unit_energys[unit]>self.sap_cost and (self.unit_has_target[unit]==3 or (self.unit_has_target[unit]==2 and self.energy_map.map[unit_pos[0],unit_pos[1]]>0)) and self.get_enemy_targets(unit_pos.tolist(), enemy_positions):
                if unit_energys[unit]>self.sap_cost and (self.unit_has_target[unit]==3 or self.unit_has_target[unit]==2) and self.get_enemy_targets(unit_pos.tolist(), enemy_positions):
                    targets = self.get_enemy_targets(unit_pos.tolist(), enemy_positions, relative=False)
                    aim = self.preaim(targets[0])
                    actions[unit]=[5,aim[0]-unit_pos[0],aim[1]-unit_pos[1]]
                else:
                    if unit_energys[unit]<self.move_cost:
                        actions[unit]=[0,0,0]
                    elif self.unit_path[unit]:
                        if self.relic_map.map_possibles[self.unit_path[unit][0][0],self.unit_path[unit][0][1]]==1:
                            if discover_flag:
                                if self.relic_map.map_possibles[unit_pos[0],unit_pos[1]]==1:
                                    actions[unit] = self.relic_map.move_away(self.tile_map.map, [unit_pos[0],unit_pos[1]])
                                    self.unit_path[unit].insert(0, unit_pos)
                                else:
                                    actions[unit]=[0,0,0]
                            else:
                                actions[unit] = [direction_to(unit_pos, self.unit_path[unit].pop(0)), 0, 0]
                                discover_flag=1
                        else:
                            actions[unit] = [direction_to(unit_pos, self.unit_path[unit].pop(0)), 0, 0]
                    else:
                        if self.relic_map.map_possibles[unit_pos[0],unit_pos[1]]==1:
                            if discover_flag:
                                actions[unit] = self.relic_map.move_away(self.tile_map.map, [unit_pos[0],unit_pos[1]])
                                self.unit_path[unit].insert(0, unit_pos)
                            else:
                                actions[unit]=[0,0,0]
                                discover_flag = 1
                        else:
                            actions[unit]=[0,0,0]
        #if step>106 and step<110:
        #    print("Step: ", step, "\n", self.unit_has_target, "\n", self.unit_targets, "\n", self.unit_path, "\n", actions)
        #print(step, "3")
        '''plt.imshow(self.relic_map.map_possibles.T)
        plt.show()
        plt.imshow(self.relic_map.map_confidence.T)
        plt.show()
        plt.imshow(self.relic_map.map_knowns.T)
        plt.show()'''
        self.previous_energys = unit_energys
        self.relic_map.map_occupied = np.zeros((24,24))
        self.prev_points = team_points[self.team_id]
        self.prev_points_increase = increase
        self.prev_actions = actions
        self.previous_positions = unit_positions
        return actions

In [ ]:
n = 50
bug = 976553406
evaluate_agents(ProxyAgent, Agent2, seed=42, games_to_play=n, render=True, model_name="models/full_test_5kk")

In [ ]:
!cd agent && tar -czf submission.tar.gz *
!mv agent/submission.tar.gz .

In [ ]:
!luxai-s3 my_agent_attack/main.py my_agent_defense/main.py --seed 42 -o replay.html